In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from osgeo import gdal

In [2]:
metadata = pd.read_csv('Data/Metadata/102GL_meta.csv')
print(metadata.columns)
metadata.head()

Index(['patch', 'Expedition', 'date [DD.MM.YYYY]', 'time [HH:MM]',
       'water_temp [°C]', 'ph [pH]', 'do [mg l-1]', 'do_sat [saturation]',
       'w_co2 [mATM]', 'conductivity [uS cm -1]', 'turb [NTU]', 'gl_name',
       'lat_sn [DD]', 'lon_sn [DD]', 'ele_sn [m]', 'lat_sp [DD]',
       'lon_sp [DD]', 'ele_sp [m]', 'sn_sp_dist [m]', 'sn_sp_ele [m]',
       'gl_sa [km2]', 'gl_cov [%]', 'gl_a [km2]', 'rgi_v6', 'glims_id',
       'chla [ug g-1]', 'eps [ugC g-1]', 'ag [nmol g-1 h-1]',
       'ap [nmol g-1 h-1]', 'bg [nmol g-1 h-1]', 'lap [nmol g-1 h-1]',
       'nag [nmol g-1 h-1]', 'bp [ngC g-1 h-1]', 'sba [cells g-1]',
       'respiration [mg 02 g-1 h-1]', 'n3_srp [ug l-1]', 'n4_nh4 [ug l-1]',
       'n5_no3 [ug l-1]', 'n6_no2 [ug l-1]'],
      dtype='object')


,patch,Expedition,date [DD.MM.YYYY],time [HH:MM],water_temp [°C],ph [pH],do [mg l-1],do_sat [saturation],w_co2 [mATM],conductivity [uS cm -1],...,bg [nmol g-1 h-1],lap [nmol g-1 h-1],nag [nmol g-1 h-1],bp [ngC g-1 h-1],sba [cells g-1],respiration [mg 02 g-1 h-1],n3_srp [ug l-1],n4_nh4 [ug l-1],n5_no3 [ug l-1],n6_no2 [ug l-1]
0,GL1_UP_1,New Zealand,22.01.19,10:00,2.4,7.81,13.3,99.9,406.3,60.1,...,0.005041,1.082781,0.000000,40.976005,5582103.22,0.003670,13.6,32.1,44.2,NaN
1,GL1_UP_2,New Zealand,22.01.19,10:00,2.4,7.81,13.3,99.9,406.3,60.1,...,0.002649,0.933340,0.000000,14.787120,2385219.31,0.016844,13.6,32.1,44.2,NaN
2,GL1_UP_3,New Zealand,22.01.19,10:00,2.4,7.81,13.3,99.9,406.3,60.1,...,0.007917,0.614265,0.011348,13.139656,1469103.36,0.031877,13.6,32.1,44.2,NaN
3,GL1_DN_1,New Zealand,22.01.19,14:26,3.1,7.71,13.4,100.1,442.6,49.4,...,0.016369,0.917580,0.000000,131.507876,3840447.42,0.002736,NaN,NaN,NaN,NaN
4,GL1_DN_2,New Zealand,22.01.19,14:26,3.1,7.71,13.4,100.1,442.6,49.4,...,0.012694,1.083179,0.000000,15.914010,1658472.02,0.005298,NaN,NaN,NaN,NaN


In [3]:
def DownloadDataBaseline():
    master_dir = os.getcwd()
    file = open('Data/Climatologies/Baseline/chelsa_baseline.txt','r').readlines()
    os.chdir('Data/Climatologies/Baseline/')
    for param in file:
        param = param.replace(' \n','')
        print(param)
        #os.system(f'wget {param}')
    os.chdir(master_dir)

In [64]:
DownloadDataBaseline()

https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/climatologies/1981-2010/bio/CHELSA_bio10_1981-2010_V.2.1.tif
https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/climatologies/1981-2010/bio/CHELSA_bio11_1981-2010_V.2.1.tif
https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/climatologies/1981-2010/bio/CHELSA_bio12_1981-2010_V.2.1.tif
https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/climatologies/1981-2010/bio/CHELSA_bio13_1981-2010_V.2.1.tif
https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/climatologies/1981-2010/bio/CHELSA_bio14_1981-2010_V.2.1.tif
https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/climatologies/1981-2010/bio/CHELSA_bio15_1981-2010_V.2.1.tif
https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/climatologies/1981-2010/bio/CHELSA_bio16_1981-2010_V.2.1.tif
https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/climatologies/1981-2010/bio/CHELSA_bio17_1981

In [4]:
import rasterio

def GetCoordinateValue(feature,lat,lon, scale, offset):
    dataset = rasterio.open(f'Data/Climatologies/Baseline/CHELSA_{feature}_1981-2010_V.2.1.tif')
    py, px = dataset.index(lon, lat)
    window = rasterio.windows.Window(px - 1//2, py - 1//2, 1, 1)
    clip = dataset.read(window=window)
    return((clip[0][0][0] * scale) + offset)

In [6]:
features = ['bio10','bio11','bio12','bio13','bio14','bio15','bio16','bio17','bio18','bio19','bio1',
            'bio2','bio3','bio4','bio5','bio6','bio7','bio8','bio9','fcf','fgd',
            'scd','swe','pr','tas','tasmin','tasmax']

features_scales = {'bio10':0.1,
                   'bio11':0.1,
                   'bio12':0.1,
                   'bio13':0.1,
                   'bio14':0.1,
                   'bio15':0.1,
                   'bio16':0.1,
                   'bio17':0.1,
                   'bio18':0.1,
                   'bio19':0.1,
                   'bio1':0.1,
                   'bio2':0.1,
                   'bio3':0.1,
                   'bio4':0.1,
                   'bio5':0.1,
                   'bio6':0.1,
                   'bio7':0.1,
                   'bio8':0.1,
                   'bio9':0.1,
                   'fcf':1,
                   'fgd':1,
                   'scd':1,
                   'swe':0.1,
                   'pr':0.1,
                   'tas':0.1,
                   'tasmin':0.1,
                   'tasmax':0.1}

features_offsets = {'bio10':-273.15,
                    'bio11':-273.15,
                    'bio12':0,
                    'bio13':0,
                    'bio14':0,
                    'bio15':0,
                    'bio16':0,
                    'bio17':0,
                    'bio18':0,
                    'bio19':0,
                    'bio1':-273.15,
                    'bio2':0,
                    'bio3':0,
                    'bio4':0,
                    'bio5':-273.15,
                    'bio6':-273.15,
                    'bio7':0,
                    'bio8':-273.15,
                    'bio9':-273.15,
                    'fcf':0,
                    'fgd':0,
                    'scd':0,
                    'swe':0,
                    'pr':0,
                    'tas':-273.15,
                    'tasmin':-273.15,
                    'tasmax':-273.15}

monthly = ['pr', 'tas', 'tasmin', 'tasmax']

In [7]:
for i, r in metadata.iterrows():
    lat = r['lat_sp [DD]']
    lon = r['lon_sp [DD]']
    month = r['date [DD.MM.YYYY]'].split('.')[1]
    for feature in features:
        if feature in monthly:
            val = GetCoordinateValue(f'{feature}_{month}', lat, lon, features_scales[feature], features_offsets[feature])
        else:
            val = GetCoordinateValue(feature, lat, lon, features_scales[feature], features_offsets[feature])
        metadata.at[i, feature] = val

In [8]:
metadata.head()

,patch,Expedition,date [DD.MM.YYYY],time [HH:MM],water_temp [°C],ph [pH],do [mg l-1],do_sat [saturation],w_co2 [mATM],conductivity [uS cm -1],...,bio8,bio9,fcf,fgd,scd,swe,pr,tas,tasmin,tasmax
0,GL1_UP_1,New Zealand,22.01.19,10:00,2.4,7.81,13.3,99.9,406.3,60.1,...,9.35,3.35,59.0,0.0,0.0,6553.5,585.7,13.15,9.45,16.55
1,GL1_UP_2,New Zealand,22.01.19,10:00,2.4,7.81,13.3,99.9,406.3,60.1,...,9.35,3.35,59.0,0.0,0.0,6553.5,585.7,13.15,9.45,16.55
2,GL1_UP_3,New Zealand,22.01.19,10:00,2.4,7.81,13.3,99.9,406.3,60.1,...,9.35,3.35,59.0,0.0,0.0,6553.5,585.7,13.15,9.45,16.55
3,GL1_DN_1,New Zealand,22.01.19,14:26,3.1,7.71,13.4,100.1,442.6,49.4,...,11.55,5.75,0.0,0.0,0.0,6553.5,524.1,15.25,11.65,18.45
4,GL1_DN_2,New Zealand,22.01.19,14:26,3.1,7.71,13.4,100.1,442.6,49.4,...,11.55,5.75,0.0,0.0,0.0,6553.5,524.1,15.25,11.65,18.45


In [9]:
metadata.to_csv('Data/Metadata/102GL_meta_clim.csv')